In [1]:
import yfinance as yf
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from modules.scalers import GroupByScaler
from sklearn.preprocessing import MaxAbsScaler
import json
import torch

from modules.environment import MultiPeriodPortfolioOptimizationEnv
from modules.architectures import MultiPeriodEIIE
from modules.models import DRLAgent

sns.set()

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
config_file_name = "config.json"
with open(config_file_name, "r") as jsonfile:
    config_data = json.load(jsonfile)

In [3]:
sp_100 = config_data["tickers"]["Europe"]["CAC_40"]
NUM_ASSETS = len(sp_100)
len(sp_100)

40

In [4]:
start_date = config_data["timeframe"]["cac40"]["start_date"]
end_date = config_data["timeframe"]["cac40"]["end_date"]
data_interval = config_data["data_interval"]
test_ratio = config_data["train_test_ratio"]
random_seed = config_data["random_state_seed"]

In [5]:
portfolio_raw_df = yf.download(tickers=sp_100, start=start_date, end=end_date, interval=data_interval)

portfolio_raw_df.fillna(method="bfill", inplace=True)
portfolio_raw_df = portfolio_raw_df.stack(level=1).rename_axis(["Date", "Ticker"]).reset_index(level=1)
portfolio_raw_df = portfolio_raw_df.drop("Adj Close", axis=1)
portfolio_raw_df.columns.name = None
portfolio_raw_df = portfolio_raw_df.reset_index()
portfolio_raw_df.Date = portfolio_raw_df.Date.astype(str)
portfolio_raw_df.columns = ["date", "tic", "close", "high", "low", "open", "volume"]
portfolio_raw_df = portfolio_raw_df[["date", "tic", "close", "high", "low", "volume"]]

df_portfolio_raw_train, df_portfolio_raw_test = train_test_split(portfolio_raw_df, test_size=test_ratio, shuffle=False, random_state=random_seed)
df_portfolio_train = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(df_portfolio_raw_train)
df_portfolio_test = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(df_portfolio_raw_test)


[*********************100%%**********************]  40 of 40 completed

2 Failed downloads:
['STM.PA', 'STLA.PA']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
/var/folders/02/rmgfx9wj0s99nn4tzfthtrnc0000gn/T/ipykernel_8408/541920902.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  portfolio_raw_df.fillna(method="bfill", inplace=True)
/var/folders/02/rmgfx9wj0s99nn4tzfthtrnc0000gn/T/ipykernel_8408/541920902.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  portfolio_raw_df = portfolio_raw_df.stack(level=1).rename_axis(["Date", "Ticker"]).reset_index(level=1)


In [6]:
# SETTING HYPERPARAMETERS
FEATURES = ["close", "high", "low", "volume"]
N = config_data["lookback_window"]
T = config_data["multi_step_horizon"]
NUM_FEATURES = len(FEATURES)
experiment_type = "EIIE_CAC40" 
NUM_ASSETS = len(portfolio_raw_df.tic.unique().tolist())
NUM_ASSETS
N, T

(24, 3)

In [7]:
train_environment = MultiPeriodPortfolioOptimizationEnv(
    df_portfolio_train,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=N,
    multi_period_horizon=T,
    features=FEATURES,
    normalize_df=None,
    is_train_mode=True,
    experiment_type=experiment_type
)

In [9]:
# set PolicyGradient parameters
model_kwargs = {
    "lr": 0.0001,
    "policy": MultiPeriodEIIE,
    "multi_period_horizon": T
}

# here, we can set EIIE's parameters
policy_kwargs = {
    "initial_features": NUM_FEATURES,
    "k_size": 5, # Size of Initial Kernel
    "time_window": N,
    "prediction_horizon": T
}

In [10]:
model = DRLAgent(train_environment).get_model("pg", device, model_kwargs, policy_kwargs)

In [11]:
print("TRAINING AGENT.....")
DRLAgent.train_model(model, episodes=35)

TRAINING AGENT.....


  0%|          | 0/35 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 470196.55805992085
Final accumulative portfolio value: 4.701965580599208
Maximum DrawDown: -0.8547375859247998
Sharpe ratio: 3.113797036089862


  3%|▎         | 1/35 [00:01<01:00,  1.77s/it]

Initial portfolio value:100000
Final portfolio value: 470203.9893860644
Final accumulative portfolio value: 4.702039893860644
Maximum DrawDown: -0.8547368186525942
Sharpe ratio: 3.1138219488405845


...

 94%|█████████▍| 33/35 [00:56<00:03,  1.72s/it]

Initial portfolio value:100000
Final portfolio value: 470420.5059260439
Final accumulative portfolio value: 4.704205059260439
Maximum DrawDown: -0.8547108914124126
Sharpe ratio: 3.114542537093286


 97%|█████████▋| 34/35 [00:58<00:01,  1.71s/it]

Initial portfolio value:100000
Final portfolio value: 470429.0025098353
Final accumulative portfolio value: 4.704290025098353
Maximum DrawDown: -0.8547099570292921
Sharpe ratio: 3.114570834946538


100%|██████████| 35/35 [00:59<00:00,  1.71s/it]


In [12]:
print("PERSISTING MODEL.....")
torch.save(model.train_policy.state_dict(), f"models/policy_{experiment_type}.pt")

PERSISTING MODEL.....


In [13]:
print("TESTING.....")
MEIIE_results = {
    "training": train_environment._asset_memory["final"],
    "test": {}
}

TESTING.....


In [14]:
test_environment = MultiPeriodPortfolioOptimizationEnv(
    df_portfolio_test,
    initial_amount=100000,
    comission_fee_pct=0.0025,
    time_window=N,
    multi_period_horizon=T,
    features=FEATURES,
    normalize_df=None,
    is_train_mode=False,
    experiment_type=experiment_type
)

In [15]:
policy = MultiPeriodEIIE(
    initial_features=NUM_FEATURES,
    k_size=5,
    time_window=N,
    prediction_horizon=T
)

In [16]:
policy.load_state_dict(torch.load(f"models/policy_{experiment_type}.pt"))

<All keys matched successfully>

In [17]:
DRLAgent.DRL_validation(model=model, test_env=test_environment, policy=policy)

Initial portfolio value:100000
Final portfolio value: 272564.5930133884
Final accumulative portfolio value: 2.725645930133884
Maximum DrawDown: -0.5260758899105635
Sharpe ratio: 3.4538030965159345


In [18]:
MEIIE_results["test"]["value"] = test_environment._asset_memory["final"]

In [21]:
drl_portfolio_performance = test_environment._asset_memory["final"][1:]
len(drl_portfolio_performance)

70

In [22]:
drl_portfolio_performance = [(x/100000) for x in drl_portfolio_performance]

In [23]:
performance_dataset = pd.read_csv("data/processed/performances_cac40.csv", index_col=0)

In [24]:
performance_dataset["DRL_EIIE"] = drl_portfolio_performance

In [25]:
performance_dataset.to_csv("data/processed/performances_cac40.csv")